In [1]:
import numpy as np
import csv
import pandas as pd
import numpy as np
import os

import numpy as np
import torch
import torch.nn as nn
import random
from torch.nn import TransformerEncoderLayer
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau

from scipy.signal import hilbert

In [2]:
data = np.load('../AL-NEGAT/Data/ABIDE2.npy',allow_pickle=True)
filename = '../AL-NEGAT/Data/ABIDEII_Composite_Phenotypic.csv'
csv2dict = pd.read_csv(filename,encoding='windows-1252').to_dict()
ID_list = np.array(list(csv2dict['SUB_ID'].values()))
FC_mat = []
T1_mat = []
lbl_arr = []
for id in ID_list:
    try:
        if data[id]['FC'].shape[0]==data[id]['T1'].shape[0]:
            FC_mat.append(data[id]['FC'])
            T1_mat.append(data[id]['T1'])
            lbl_arr.append(data[id]['label'])
    except:
        pass

In [3]:
FC_mat = np.array(FC_mat)
T1_mat = np.array(T1_mat)
lbl_arr = np.array(lbl_arr)

In [4]:
FC_mat.shape, T1_mat.shape, lbl_arr.shape

((546, 400, 400), (546, 400, 4), (546,))

In [5]:
lbl_arr[lbl_arr==2] = 0
lbl_arr.mean()

0.46886446886446886

In [6]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        z, x, y = self.data[idx]
        z = torch.from_numpy(z.astype(np.float32))
        x = torch.from_numpy(x.astype(np.float32))
        edge_indx = torch.tensor(np.indices((400,400)).reshape(2,-1),dtype=torch.long).t().contiguous()
        iden_mat = torch.eye(400,dtype=torch.float32)
        y = torch.tensor(y)
        return z,x,edge_indx,iden_mat,y

In [17]:
#random number generator for train test split
rng = np.random.RandomState(42)
indices = np.arange(len(FC_mat))
rng.shuffle(indices)
FC_mat = FC_mat[indices]
T1_mat = T1_mat[indices]
lbl_arr = lbl_arr[indices]

data_dict = {}
for i in range(len(FC_mat)):
    data_dict[i] = [FC_mat[i], T1_mat[i], lbl_arr[i]]

train_data = CustomDataset(list(data_dict.values())[:int(0.8*len(data_dict))])
test_data = CustomDataset(list(data_dict.values())[int(0.8*len(data_dict)):])

train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)

In [18]:
from torch_geometric.nn.conv import GCNConv

class NEGA_block(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.node_gcn = GCNConv(4,4)
        self.edge_gcn = GCNConv(400,400)
        self.softmax = nn.Softmax(dim=1)
        #batchnorm with relu
        self.batchnorm_relu = nn.Sequential(
            nn.BatchNorm1d(400),
            nn.ReLU()
        )
        self.W_z = nn.Parameter(torch.randn(400,400))
        self.W_x = nn.Parameter(torch.randn(4,4))

    def forward(self, z,x,edge_indx,iden_mat):
        
        print('Red was here')
        v = self.edge_gcn(x = iden_mat ,edge_index=edge_indx ,edge_weight= torch.flatten(z,1,-1))#400x400
        print('Red survived')
        qk = self.node_gcn(x,edge_indx)#400x4
        A = self.softmax(qk@qk.T)*v

        Z_lp1 = self.batchnorm_relu(A@z@self.W_z)+z
        X_lp1 = self.batchnorm_relu(A@x@self.W_x)+x
        return Z_lp1, X_lp1

In [19]:
#create 5 layers of NEGA blocks and flatten the output and apply mlp individually on node and edge features and concatenate them and apply mlp on the concatenated features for classification
class NEGA(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.nega1 = NEGA_block()
        self.nega2 = NEGA_block()
        self.nega3 = NEGA_block()
        self.nega4 = NEGA_block()
        self.nega5 = NEGA_block()
        self.mlp_node = nn.Sequential(
            nn.Linear(4,2),
            nn.ReLU(),
            nn.Linear(2,1)
        )
        self.mlp_edge = nn.Sequential(
            nn.Linear(400,200),
            nn.ReLU(),
            nn.Linear(200,1)
        )
        self.mlp_concat = nn.Sequential(
            nn.Linear(2,1),
            nn.ReLU(),
            nn.Linear(1,1)
        )
    def forward(self, z,x,edge_indx,iden_mat):
        z1,x1 = self.nega1(z,x,edge_indx,iden_mat)
        z2,x2 = self.nega2(z1,x1,edge_indx,iden_mat)
        z3,x3 = self.nega3(z2,x2,edge_indx,iden_mat)
        z4,x4 = self.nega4(z3,x3,edge_indx,iden_mat)
        z5,x5 = self.nega5(z4,x4,edge_indx,iden_mat)

        node_out = self.mlp_node(x5)
        edge_out = self.mlp_edge(z5)
        concat_out = self.mlp_concat(torch.cat((node_out,edge_out),dim=1))
        return concat_out

In [20]:
#train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NEGA().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)
model.train()
for epoch in range(100):
    for i, (z,x,edge_indx,iden_mat,y) in enumerate(train_loader):
        z = z.to(device)
        x = x.to(device)
        edge_indx = edge_indx.to(device)
        iden_mat = iden_mat.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        out = model(z,x,edge_indx,iden_mat)
        loss = criterion(out,y.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
    print('Epoch: ',epoch,' Loss: ',loss.item())
    scheduler.step(loss.item())

Red was here


IndexError: index 1 is out of bounds for dimension 0 with size 1